# Ref- qualified member functions

C++11 introduced reference qualifiers to class non-static member functions. Before C++11, a member function could be `const` or not-`const`, where the specified CV qualification of the member function applied to the calling instance.  

In [ ]:
#pragma cling add_include_path("/p/project/training2444/local/include")
#include <Vbose.hh>

In [ ]:
namespace v0 {
struct FileData {
    Vbose header_text{};
    unsigned long bulk{};
    auto header() const -> const Vbose& { return header_text; }
    auto header() -> Vbose& { return header_text; }
};

auto readfile(std::string fn) -> FileData
{
    static auto c = 0UL;
    return {{fn}, c++};
}
}

Two example uses are shown below.

In [ ]:
{
    auto f1 = v0::readfile("f1");
    auto h1 = f1.header(); // copy
    auto h2 = v0::readfile("f2").header();
    // move desirable, but h2 is copy constructed.
}

Both `h1` and `h2` are copy constructed above, because even though the second call to `header` is invoked on a temporary object, that information isn't used in the definition of `header`. It still returns an L-value reference. How would we write these functions specially for those situations where the calling instance is in fact a temporary?

In [ ]:
namespace v1 {
struct FileData {
    Vbose header_text{};
    unsigned long bulk{};
// ref qualified members, since C++11
    auto header() const& -> const Vbose& { return header_text; }
    auto header() & -> Vbose& { return header_text; }
    auto header() const&& -> const Vbose&& { return std::move(header_text); }
    auto header() && -> Vbose&& { return std::move(header_text); }
};

auto readfile(std::string fn) -> FileData
{
    static auto c = 0UL;
    return {{fn}, c++};
}
}

In [ ]:
{
    auto f1 = v1::readfile("f1");
    auto h1 = f1.header(); // copy
    auto h2 = v1::readfile("f2").header(); // move
}

We have the desired effect of using possible optimisations when the calling instance is a temporary or an expiring value.